# Benchmarking

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Pre-requirements" data-toc-modified-id="Pre-requirements-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Pre-requirements</a></span></li><li><span><a href="#Info" data-toc-modified-id="Info-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Info</a></span></li><li><span><a href="#Helper" data-toc-modified-id="Helper-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Helper</a></span></li><li><span><a href="#DBSCAN-vs.-CNN-scikit-learn-implementation" data-toc-modified-id="DBSCAN-vs.-CNN-scikit-learn-implementation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>DBSCAN vs. CNN scikit-learn implementation</a></span></li><li><span><a href="#Fit-variants" data-toc-modified-id="Fit-variants-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Fit variants</a></span><ul class="toc-item"><li><span><a href="#From-neighbours" data-toc-modified-id="From-neighbours-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>From neighbours</a></span></li></ul></li></ul></div>

## Pre-requirements

In [1]:
import importlib
from operator import itemgetter
import sys
import time


from IPython.core.display import display, HTML
import numpy as np
%matplotlib widget
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn import cluster
from sklearn import datasets
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

import cfits
from core import cnn
from snippets import fits

%load_ext Cython
%load_ext line_profiler
%load_ext memory_profiler

In [38]:
importlib.reload(cnn)

<module 'core.cnn' from '/home/janjoswig/CNN/core/cnn.py'>

In [12]:
importlib.reload(fits)

<module 'snippets.fits' from '/home/janjoswig/CNN/tests/benchmark/snippets/fits.py'>

In [2]:
# Matplotlib configuration
mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rc_file("../../tutorial/matplotlibrc", use_default_template=False)

In [3]:
# Jupyter notebook configuration
display(HTML("<style>.container { width:90% !important; }</style>"))

## Info

In [4]:
print(sys.version)

3.8.1 (default, Jan 23 2020, 07:59:15) 
[GCC 8.3.0]


## Helper

In [41]:
def plot_data(data, labels=None, ax=None, noise=0):
    if ax is None:
        plt.close('all')
        fig, ax = plt.subplots(
            figsize=(
                mpl.rcParams["figure.figsize"][0] / 2,
                mpl.rcParams["figure.figsize"][1]
                )
            )
    else:
        fig = ax.get_figure()
        
    if labels is None:
        ax.plot(
            *data.T,
            linestyle="",
            color="None",
            marker="o",
            markersize=4,
            markerfacecolor="gray",
            markeredgecolor="k",
            )

    else:
        ax.plot(
            *data[np.where(labels == noise)[0]].T,
            linestyle="",
            color="None",
            marker="o",
            markersize=4,
            markerfacecolor="gray",
            markeredgecolor="k",
            )

        for cluster_number in range(noise + 1 , np.max(labels) + 1):
            ax.plot(
                *data[np.where(labels == cluster_number)[0]].T,
                linestyle="",
                marker="o",
                markersize=4,
                markeredgecolor="k",
                )

    ax.set(**{
        "xticks": (),
        "yticks": (),
        "xticklabels": (),
        "yticklabels": (),
        "aspect": "equal"
        })

## DBSCAN vs. CNN scikit-learn implementation

In [39]:
np.random.seed(42)

In [5]:
# Data set generation
# circles
def gen_circles(n):
    noisy_circles, _ = datasets.make_circles(
        n_samples=n,
        factor=.5,
        noise=.05,
        random_state=10
        )
    
    return StandardScaler().fit_transform(noisy_circles)

gen_circles.label = "circles"

# blobs                            
def gen_blobs(n):
    blobs, _ = datasets.make_blobs(
        centers=[[-10, -10], [10, -10], [10, 10]],
        n_samples=n,
        random_state=10
    )
    return StandardScaler().fit_transform(blobs)

gen_blobs.label = "blobs"

In [38]:
plot_data(gen_circles(1000))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [42]:
data = gen_circles(5000)
labels_dbscan = cluster.dbscan(data, eps=0.2, min_samples=5)[1]
labels_cnn = cluster.cnn(data, eps=0.25, min_samples=2)

plt.close('all')
fig, Ax = plt.subplots(
    1, 2,
    figsize=(
        mpl.rcParams["figure.figsize"][0],
        mpl.rcParams["figure.figsize"][1]
        )
    )
Ax = Ax.flatten()

for i, labels in enumerate([labels_dbscan, labels_cnn]):
    plot_data(data, labels=labels, ax=Ax[i], noise=-1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
data = gen_circles(5000)
cobj = cnn.CNN(data)
cobj.calc_dist()
cobj.calc_neighbours_from_dist(0.25)
cobj.fit_generic(radius_cutoff=0.25, cnn_cutoff=2, max_clusters=None, get_neighbours=cobj.get_neighbours_lookup)
labels = np.asarray(cobj.labels)

plot_data(data, labels=labels)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Fit variants

In [16]:
importlib.reload(fits)

<module 'snippets.fits' from '/home/janjoswig/CNN/tests/benchmark/snippets/fits.py'>

In [6]:
report_dir = "reports/T460"

### From neighbours

In [7]:
def prepare_neighbours(n, r, gen_fxn, numpy=False):
    data = gen_fxn(n)
    neighbour_model = NearestNeighbors(radius=r).fit(data)
    neighbours = neighbour_model.radius_neighbors(data, return_distance=False)
    neighbours = [set(x) for x in neighbours]  # Remove self-counting 
    for c, s in enumerate(neighbours):
        s.remove(c)
    if numpy:
        neighbours = np.array([np.array([y for y in x]) for x in neighbours])
        
    return neighbours

In [17]:
n = 10000
r = 0.25
gen_fxn = gen_circles
ds = gen_fxn.label
neighbours = prepare_neighbours(n, r, gen_fxn)

In [8]:
class DS:
    def __init__(self, gen_fxn, n, gen_fxn_args=None, gen_fxn_kwargs=None):
        self.gen_fxn = gen_fxn
        self.n = n
        
        if gen_fxn_kwargs is None:
            gen_fxn_kwargs = {}
        if gen_fxn_args is None:
            gen_fxn_args = ()  
        
        self.data = gen_fxn(n, *gen_fxn_args, **gen_fxn_kwargs)
        self.neighbours = None
        self.r = None
        self.timeits = {}
        
    def prepare_neighbours(self, r, numpy=False):
        neighbour_model = NearestNeighbors(radius=r).fit(self.data)
        neighbours = neighbour_model.radius_neighbors(
            self.data, return_distance=False
            )
        neighbours = [set(x) for x in neighbours]  # Remove self-counting 
        for c, s in enumerate(neighbours):
            s.remove(c)
        if numpy:
            neighbours = np.array([np.array([y for y in x]) for x in neighbours])    
        
        self.neighbours = neighbours
        self.r = r
        
    def __str__(self):
        try:
            desc_gen = self.gen_fxn.label
        except AttributeError:
            desc_gen = self.gen_fxn.__name__
            
        # p: from points
        # d: from distances
        # n: from neighbours
        
        if self.neighbours is not None:
            desc_from = f"n{self.r}"
        elif self.dist is not None:
            desc_from = f"d"
        else:
            desc_from = f"p"
            
        return f"{desc_gen}_{self.n}_{desc_from}"
    
    
    def ratios(self, base=None):
        if base is not None:
            base = self.timeits[base].average
        else:
            base = min(x.average for x in self.timeits.values())
            
        return sorted([
            (k, v.average / base)
            for k, v in self.timeits.items()
            ], key=itemgetter(1))
        

In [ ]:
def ratios(self, base=None):
    if base is not None:
        base = self.timeits[base].average
    else:
        base = min(x.average for x in self.timeits.values())

    return sorted([
        (k, v.average / base)
        for k, v in self.timeits.items()
        ], key=itemgetter(1))

In [37]:
def profile_fxn(
        f, ds, report_dir, *args,
        t=True, l=True, m=True, label=None, **kwargs):
    """Function profiling procedure
    
    Runs %lprun and %timeit line magic on a globally defined function
    `fxn`.  Function args and kwargs need to be defined globally as
    well.  This is necessary, because (at least lprun) line magic does
    not seem to work well with local variables, e.g. the following
    alternative did not work (raises `NameError`): 
    
       def profile_fxn(fxn, *args, **kwargs):
           %lprun -f fxn fxn(*args, **kwargs)
           ...
    
    This function expects a :obj:`DS` object, providing a dataset and
    pre-calculated values if necessary.  Report details are deduced from
    this object.  Timings are saved to the object.
    
    Args:
       f (:obj:`func`): Function to profile.
       ds (:obj:`DS`): Data set object.
       report_dir (str): Output directory file path.
       *args: Arguments passed to `f`
    
    Keyword args:
       label (optional, str): Label to identify the run.  If `None`,
          `fxn.__name__` is used.
       *kwargs: Keyword arguments passed to `f`.
          
    
    Returns:
       None
    
    """
    
    global fxn
    global fxn_args
    global fxn_kwargs
    
    fxn = f
    fxn_args = args
    fxn_kwargs = kwargs
    
    if l:
        %lprun -T {report_dir}/{fxn.__name__}_{ds.__str__()}.lprun -f fxn fxn(*fxn_args, **fxn_kwargs)
    if t:
        o = %timeit -o fxn(*fxn_args, **fxn_kwargs)

        if label is None:
            label = fxn.__name__
        ds.timeits.update({label: o})

In [56]:
ds = DS(gen_circles, 100)
ds.prepare_neighbours(0.25)

In [42]:
ds.prepare_neighbours(0.25)

In [13]:
print(ds)

circles_10000_n0.25


In [23]:
# Implementation using original implementation
profile_fxn(
    fits.fit_from_neighbours_baseline,
    ds, report_dir,
    1, ds.neighbours,  # function args
    label="baseline"
    )


*** Profile printout saved to text file 'reports/T460/fit_from_neighbours_baseline_circles_10000_n0.25.lprun'. 
2.79 s ± 31.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [43]:
# Implementation using only standard library
profile_fxn(
    fits.fit_stdlib_from_neighbours_index,
    ds, report_dir,
    1, ds.neighbours,  # function args
    label="std_index"
    )


*** Profile printout saved to text file 'reports/T460/fit_stdlib_from_neighbours_index_circles_10000_n0.25.lprun'. 
404 ms ± 9.17 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
# Implementation using only standard library
profile_fxn(
    fits.fit_stdlib_from_neighbours_loop,
    ds, report_dir,
    1, ds.neighbours,  # function args
    label="std_loop"
    )


*** Profile printout saved to text file 'reports/T460/fit_stdlib_from_neighbours_loop_circles_10000_n0.25.lprun'. 
403 ms ± 11.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [45]:
for x in ds.ratios():
    print(f"{x[0]:>15}: {x[1]:7.3f}")

cython_numpy_loop:   1.000
   numpy_filter:   1.005
       std_loop:   1.076
      std_index:   1.079
    numpy_index:   2.539
     numpy_loop:   2.804


In [59]:
ds = DS(gen_circles, 100)
ds.prepare_neighbours(0.25, numpy=True)

In [47]:
ds.prepare_neighbours(0.25, numpy=True)

In [16]:
# Implementation using numpy
profile_fxn(
    fits.fit_numpy_from_neighbours_index,
    ds, report_dir,
    1, ds.neighbours,  # function args
    label="numpy_index"
    )


*** Profile printout saved to text file 'reports/T460/fit_numpy_from_neighbours_index_circles_10000_n0.25.lprun'. 
951 ms ± 41.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
# Implementation using numpy
profile_fxn(
    fits.fit_numpy_from_neighbours_loop,
    ds, report_dir,
    1, ds.neighbours,  # function args
    label="numpy_loop"
    )


*** Profile printout saved to text file 'reports/T460/fit_numpy_from_neighbours_loop_circles_10000_n0.25.lprun'. 
1.05 s ± 85.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
# Implementation using numpy
profile_fxn(
    fits.fit_numpy_from_neighbours_filtermembers,
    ds, report_dir,
    1, ds.neighbours,  # function args
    label="numpy_filter"
    )


*** Profile printout saved to text file 'reports/T460/fit_numpy_from_neighbours_filtermembers_circles_10000_n0.25.lprun'. 
376 ms ± 3.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
# Implementation using cythonised numpy
profile_fxn(
    cfits.cfit_from_neighbours,
    ds, report_dir,
    1, ds.neighbours,  # function args
    label="cython_numpy_loop",
    l=False
    )

375 ms ± 5.47 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [72]:
ds.prepare_neighbours(0.25)

In [75]:
%timeit fits.fit_stdlib_from_neighbours_loop(1, ds.neighbours)

60.6 µs ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [67]:
cfit_from_neighbours_(1, ds.neighbours)

0
1
2
3
4
69
29
27
18
43
12
84
97
5
6
81
48
45
71
7
8
9
10
11


ValueError: Buffer dtype mismatch, expected 'npy_intp' but got 'double'

In [86]:
%timeit cfit_stdlib_from_neighbours_loop(1, ds.neighbours)

36.3 µs ± 408 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [35]:
importlib.reload(cfits)

<module 'cfits' from '/home/janjoswig/CNN/cfits.cpython-38-x86_64-linux-gnu.so'>

In [85]:
%%cython --annotate

# distutils: language = c++
# cython: infer_types = True
# cython: boundcheck = False
# cython: wraparound = False

from collections import deque
cimport cython

def cfit_stdlib_from_neighbours_loop(
        int cnn_cutoff,
        neighbours: List[Set[int]]) -> List:
    """Worker function variant applying the CNN algorithm.

    Assigns labels to points starting from pre-computed neighbour list.
    Uses Python standard library only.  Minor design difference to
    `fit_stdlib_from_neighbours_index` in looping over all points
    instead of checking for `include.index(True)`. Expected to be faster
    when many cluster initialisations are done. Also, checking queue
    at the end of the inner loop instead of in the begenning (no
    performance difference expected, but unnecessary adding of initial
    points to queue is avoided).

    Args:
        cnn_cutoff: Similarity criterion
        neighbours: List of length #points containing sets of
            neighbouring point indices

    Returns:
        Labels
    """
    
    cdef Py_ssize_t point

    cdef Py_ssize_t len_ = len(neighbours)

    # Initialise labels
    labels = [0 for _ in range(len_)]

    # Track assigment
    consider = [True for _ in range(len_)]

    # Start with first cluster (0 = noise)
    cdef Py_ssize_t current = 1

    # Initialise queue of points to scan
    queue = deque()

    for point in range(len_):
        if not consider[point]:
            # Point already assigned
            continue
        labels[point] = current          # Assign cluster label
        consider[point] = False           # Mark point as included

        while True:
            # Loop over neighbouring points
            neigh = neighbours[point]
            for member in neigh:
                if not consider[member]:
                    # Point already assigned
                    continue

                # conditional growth
                if len(neigh.intersection(neighbours[member])) >= cnn_cutoff:
                    labels[member] = current
                    consider[member] = False
                    queue.append(member)

            if not queue:
                break
            point = queue.popleft()  # FIFO

        current += 1

    return labels

In [66]:
%%cython  --annotate

# distutils: language = c++

cimport cython
from libcpp.vector cimport vector
cimport numpy as np
import numpy as np


@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing
def cfit_from_neighbours_(
        int cnn_cutoff,
        np.ndarray[object, ndim=1] neighbours):
    """Worker function variant applying the CNN algorithm.

    Assigns labels to points starting from pre-computed neighbour list.
    Cythonised.

    Args:
        cnn_cutoff: Similarity criterion
        neighbours: NumPy array of length #points containing arrays of
            neighbouring point indices

    Returns:
        Labels
    """

    cdef np.npy_intp point, current, m, member, len_
    cdef np.ndarray[np.npy_intp, ndim=1] neigh
    cdef vector[np.npy_intp] labels, stack  # LIFO

    len_ = neighbours.shape[0]

    # Initialise labels
    labels = np.zeros(len_)

    # Track assigment
    consider = np.ones(len_, dtype=bool)

    # Start with first cluster (0 = noise)
    current = 1

    for point in range(len_):
        if not consider[point]:
            continue

        labels[point] = current  # Assign cluster label
        consider[point] = False  # Mark point as included

        while True:
            print(point)
            # Loop over neighbouring points
            neigh = neighbours[point]
            # loop over members not included
            for m in range(neigh.shape[0]):
                member = neigh[m]
                if not consider[member]:
                    continue
                # conditional growth
                if len(np.intersect1d(
                        neigh, neighbours[member], assume_unique=True
                        )) >= cnn_cutoff:
                    labels[member] = current
                    consider[member] = False
                    stack.push_back(member)

            if stack.size() == 0:
                break
            point = stack.back()  # LIFO
            stack.pop_back()

        current += 1

    return labels

In [23]:
%%cython --annotate

# distutils: language = c++

cimport cython
from libcpp.vector cimport vector
cimport numpy as np
import numpy as np


@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing
def cfit_from_neighbours(
        int cnn_cutoff,
        np.ndarray[object, ndim=1] neighbours):
    """Worker function variant applying the CNN algorithm.

    Assigns labels to points starting from pre-computed neighbour list.
    Cythonised.

    Args:
        cnn_cutoff: Similarity criterion
        neighbours: NumPy array of length #points containing arrays of
            neighbouring point indices

    Returns:
        Labels
    """

    cdef np.npy_intp point, m_, member, len_ = neighbours.shape[0]
    cdef np.ndarray[np.npy_intp, ndim=1] neigh
    cdef vector[np.npy_intp] stack  # LIFO
    stack.reserve(len_)
    
    # Initialise labels
    cdef np.ndarray[np.uint32_t, ndim=1] labels = np.zeros(len_, dtype=np.uint32)

    # Track assigment
    cdef np.ndarray[np.uint8_t, ndim=1, mode='c'] consider = np.ones(len_, dtype=bool)

    # Start with first cluster (0 = noise)
    cdef unsigned int current = 1

    for point in range(len_):
        if not consider[point]:
            continue

        labels[point] = current  # Assign cluster label
        consider[point] = False  # Mark point as included

        while True:
            # Loop over neighbouring points
            neigh = neighbours[point]
            # loop over members not included
            for member in neigh:
                # member = neigh[m_]
                # conditional growth
                if len(np.intersect1d(
                        neigh, neighbours[member], assume_unique=True
                        )) >= cnn_cutoff:
                    labels[member] = current
                    consider[member] = False
                    stack.push_back(member)

            if stack.size() == 0:
                break
            point = stack.back()  # LIFO
            stack.pop_back()

        current += 1

    return labels

In [39]:
cfit(1, neighbours)

NameError: name 'cfit' is not defined

In [41]:
%%cython --annotate

def f():
    for i in range(10):
        print(i)
